# Week 3 assignment (consists of 3 parts, please see below)

## Part 1

In [1]:
import requests
import pandas as pd

###  Assign wiki url to variable

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

###  Get table into variable

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

### Start reading table raws into arrays

In [4]:
links=My_table.findAll('th')

Postalcode=[]
Borough=[]
Neighbourhood=[]

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        Postalcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))

### Create dataframe and assign appropriate arrays to columns

In [5]:
df=pd.DataFrame()
df['Postalcode']= Postalcode
df['Borough']= Borough
df['Neighbourhood']= Neighbourhood
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Discard 'Not assigned' values from Borough

In [6]:
df.drop(df.loc[df['Borough'] == 'Not assigned'].index, inplace=True)
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Merge raws with same postalcode

In [7]:
df=df.sort_values(by=['Postalcode','Neighbourhood'], ascending=False)
df2 = (df.groupby(['Postalcode','Borough'])['Neighbourhood']
       
       .apply(lambda x: list(set(x)))
       .reset_index())
df2=df2.sort_values(by=['Postalcode'])

#convert list to string values
for i in df2['Neighbourhood'].index:
    df2['Neighbourhood'][i] =','.join(df2['Neighbourhood'][i])
    df2['Neighbourhood'][i] = df2['Neighbourhood'][i].replace('\n', '');
    
df2.head()


,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Replace  neighborhood  'Not assigned' values by  borough values

In [8]:
df2.loc[df2['Neighbourhood'] == 'Not assigned','Neighbourhood']=df2['Borough']


### Number of rows:

In [9]:
df2.shape

(103, 3)

## Part 2 of week 3 assignment

### Importing cordinates from csv

In [10]:
url='http://cocl.us/Geospatial_data'
c=pd.read_csv(url)
c.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


removing space in column name'Postal Code'

In [11]:
c.rename(columns={'Postal Code':'Postalcode'},inplace=True)
c.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
c.shape

(103, 3)

### Merging postal code, neighbourhoods dataframe with cordinates dataframe

In [13]:
df_c=pd.merge(df2, c, on='Postalcode', how='inner')
df_c.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3 of week 3 assignment

## Filterig dataframe by boroughs that contain the word Toronto

In [14]:
df_to=df2[df2['Borough'].str.contains('Toronto')]
df_to.reset_index()
df_to.shape

(38, 3)

## Clustering the neighborhoods of Toronto

In [16]:
df_to.head()

,Postalcode,Borough,Neighbourhood
37,M4E,East Toronto,The Beaches
41,M4K,East Toronto,"Riverdale,The Danforth West"
42,M4L,East Toronto,"India Bazaar,The Beaches West"
43,M4M,East Toronto,Studio District
44,M4N,Central Toronto,Lawrence Park
